In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [ ]:
len(tf.config.list_physical_devices('GPU'))

In [ ]:
def process_csv_file(file):
    try:
     csv_data = pd.read_csv(file)
    except:
         print("CANT READ CSV: ", file)
         return
    
    if csv_data.isnull().values.any():
            
            return False
    try:
        csv_data = csv_data.drop("class", axis = 1)
    except KeyError:
        pass
    return (csv_data.to_numpy())

In [ ]:
!cd /Tmp/ASL-data/Database-augmented/coffee/; ls;
!ls

In [ ]:
file = "/Tmp/ASL-data/Database-augmented/test/coffee/2sGQuduhAf41354.csv"
process_csv_file(file)

In [ ]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [ ]:
def prepare_data(folder_path, labels, train=True):
    total = 0
    max_length = 211
    
    # find max length
    # for folder in os.listdir(folder_path):
    #     print(folder)
    #     if folder != ".DS_Store":
    #         total += len(os.listdir(os.path.join(folder_path, folder)))
    #         for file in os.listdir(os.path.join(folder_path, folder)):
    #             if file != ".DS_Store":
    #                 data = process_csv_file(os.path.join(folder_path, folder, file))
    #                 if data is not(False):
    #                     if data.shape[0] > max_length:
    #                         max_length = data.shape[0]
    # print(max_length)
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    print("----------")
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            
            i=0
            for file in os.listdir(os.path.join(folder_path, folder)):
                if file != ".DS_Store":
                    data = process_csv_file(os.path.join(folder_path, folder, file))
                    if data is not(False):
                        data = set_padding(data, max_length)
                        X = np.append(X, data, axis=0)
                        Y = np.append(Y, labels[folder])
                        if (i% 50 == 0):
                            print(i)
                    i+=1
                if train == True:
                    if i == 20:
                        break
                elif i == 21: # test dataset limit
                    break
    print(total, max_length)
    return X, Y

In [ ]:
folder_path_train = "/Tmp/ASL-data/Database-augmented/train"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_train, Y_train = prepare_data(folder_path_train, labels)

In [ ]:
folder_path_test = "/Tmp/ASL-data/Database-augmented/test"
labels = {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_test, Y_test = prepare_data(folder_path_test, labels, train=False)

In [ ]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

In [ ]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T
Y

In [ ]:
# shuffle data
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_val,Y_val = shuffle(X_test, Y_test, random_state=0)
X_train, Y_train

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
x_shape = X_train.shape

In [ ]:
from tensorflow.keras import layers
masking_input = []
for i in range(x_shape[0]):
    masking_input.append(X_train[i].T[0])

masking_input = np.array(masking_input)

masking_input.shape

In [ ]:
embedding = layers.Embedding(input_dim=x_shape[1], output_dim=x_shape[1], mask_zero=True)
masked_output = embedding(masking_input)

print(masked_output._keras_mask)
masked_output
# masking_layer = layers.Masking()
# # Simulate the embedding lookup by expanding the 2D input to 3D,
# # with embedding dimension of 10.
# unmasked_embedding = tf.cast(
#     tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
# )

# masked_embedding = masking_layer(unmasked_embedding)
# print(masked_embedding._keras_mask)

In [ ]:
#LSTM model 
from tensorflow.keras import layers
x_shape = (457, 211, 300)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(211, 300)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64))
model_lstm.add(Flatten())
model_lstm.add(Dense(128, activation="relu"))
model_lstm.add(Dense(64, activation="relu"))
model_lstm.add(Dense(4, activation='softmax'))


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "/Tmp/shariffa/LSTM-4-labels.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [ ]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_lstm.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val), callbacks=[checkpoint])

In [ ]:
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.GRU(64)(inputs, mask=masked_output._keras_mask)
# x = layers.LSTM(64)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model_lstm2= tf.keras.Model(inputs, outputs)
model_lstm2.summary()

In [ ]:
model_lstm2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)